# GEMINI PROMPT


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import json
import re
import time


json_folder = "/content/drive/MyDrive/LMM/json/new_prompt"


# test_dataset = "/content/drive/MyDrive/LMM/dataset/test"

# pdf_names = {
#     os.path.splitext(f)[0]
#     for f in os.listdir(test_dataset)
#     if f.lower().endswith(".pdf")
# }
# json_paths = [
#     os.path.join(json_folder, f)
#     for f in os.listdir(json_folder)
#     if f.lower().endswith(".json") and os.path.splitext(f)[0] in pdf_names
# ]

In [ ]:
json_full_paths = [os.path.join(json_folder, f) for f in os.listdir(json_folder) if f.lower().endswith(".json")]

In [ ]:
len(json_full_paths)

In [ ]:
first_json_path = json_full_paths[0]
with open(first_json_path, "r", encoding="utf-8") as f:
    first_json_content = json.load(f)
demo = json.dumps(first_json_content, indent=2)
print(demo)

In [ ]:
!pip install -q "google-genai>=1.16.0"

In [ ]:
from google import genai
client = genai.Client(api_key="YOUR_API_KEY_HERE")
model = "gemini-2.5-flash-lite"

In [ ]:
prompt = f"""
You are an expert in tropical and infectious diseases.
You obtain information from authoritative public health sources.
You will be provided with symptoms and contextual information such as
age, gender, location, pre-existing conditions, and lifestyle factors.
Please respond to the questions with a single answer on the most likely diagnosis.
- Provide  diagnosis with evidence
- Do not be verbose

Here is the information about the case:
\"\"\"
{demo}
\"\"\"

Here are some example questions with appropriate answers:

prompt: Patient is a 28-year-old male reporting with fever, severe headaches, irritability, fatigue, and aching muscles and joints.
They have developed a skin rash, show signs of confusion, and report uncharacteristic sleeping during the day.
Patient lives in the Uige province and owns domestic animals.
diagnosis: Sleeping sickness

prompt: Patient is a 49-year-old female reporting with fever, headache, muscle pain, sore throat, loss of appetite, diarrhea, and vomiting.
They are also hemorrhaging. Patient is a health worker from Vuhovi.
Patient has a family member who was recently admitted with similar symptoms.
diagnosis: Ebola

OUTPUT FORMAT (valid JSON only):
{{
  "prompt": "{{input_prompt}}",
  "diagnosis": ""
}}
"""
response = client.models.generate_content(
    model=model,
    contents=prompt
)

qa_text = response.text.strip()
qa_text = re.sub(r"^```json|```$", "", qa_text).strip()
print(qa_text)

In [ ]:

results = []
for chunk in json_full_paths:
    with open(chunk, "r", encoding="utf-8") as f:
        data = json.load(f)

    full_text = json.dumps(data, indent=2, ensure_ascii=False)

    prompt = f"""
You are an expert in tropical and infectious diseases.
You obtain information from authoritative public health sources.
You will be provided with symptoms and contextual information such as
age, gender, location, pre-existing conditions, and lifestyle factors.
Please respond to the questions with a single answer on the most likely diagnosis.
- Provide  diagnosis with evidence
- Do not be verbose

Here is the information about the case:
\"\"\"
{full_text}
\"\"\"

Here are some example questions with appropriate answers:

prompt: Patient is a 28-year-old male reporting with fever, severe headaches, irritability, fatigue, and aching muscles and joints.
They have developed a skin rash, show signs of confusion, and report uncharacteristic sleeping during the day.
Patient lives in the Uige province and owns domestic animals.
diagnosis: Sleeping sickness

prompt: Patient is a 49-year-old female reporting with fever, headache, muscle pain, sore throat, loss of appetite, diarrhea, and vomiting.
They are also hemorrhaging. Patient is a health worker from Vuhovi.
Patient has a family member who was recently admitted with similar symptoms.
diagnosis: Ebola

OUTPUT FORMAT (valid JSON only):
{{
  "prompt": "{{input_prompt}}",
  "diagnosis": ""
}}
"""
    response = client.models.generate_content(
        model=model,
        contents=prompt
    )

    qa_text = response.text.strip()
    qa_text = re.sub(r"^```json|```$", "", qa_text).strip()
    try:
        qa = json.loads(qa_text)
    except json.JSONDecodeError:
        print(f"Could not parse JSON for file {chunk}.")
        continue
    qa["file_name"] = chunk.split("/")[-1]
    results.append(qa)
    print(qa)
    time.sleep(5)


In [ ]:
# import json
# import re
# import time

# results = []

# for chunk in json_full_paths:
#     # Load JSON file
#     with open(chunk, "r", encoding="utf-8") as f:
#         data = json.load(f)

#     full_text = json.dumps(data, indent=2, ensure_ascii=False)

#     # Prepare the prompt for Gemini
#     prompt = f"""
# You are an expert in tropical and infectious diseases. You obtain information from authoritative public health sources.
# You will be provided a question with symptoms, and contextual information such as age, gender, location, pre-existing conditions and lifestyle factors.
# You will also be provided with multiple choice options. Please respond to the questions with a single answer on the most likely diagnosis.

# - Only provide the best answer with the letter followed by the diagnosis.
# - DO NOT include any other context.
# - DO NOT generate new questions.

# INPUT TEXT:
# \"\"\"
# {full_text}
# \"\"\"

# Here are some example questions with appropriate answers.

# prompt: Patient is a 28 year old male reporting with fever, severe headaches, irritability, fatigue and aching muscles and joints.
# They have developed a skin rash, show signs of confusion and report uncharacteristic sleeping during the day.
# Patient lives in the Uige province and owns domestic animals.
# (A) Yellow Fever
# (B) Cholera
# (C) Sleeping sickness
# (D) Malaria
# (E) None of the above
# answer:(C) Sleeping sickness

# prompt: Patient is a 49 year old female reporting with fever, headache, muscle pain, sore throat, loss of appetite, diarrhea and vomiting.
# They are also hemorrhaging. Patient is a health worker from Vuhovi.
# Patient has a family member who was recently admitted with similar symptoms.
# (A) Cholera
# (B) Bullous Impetigo
# (C) COVID
# (D) Sleeping sickness
# (E) Ebola
# answer:(E) Ebola

# OUTPUT FORMAT (valid JSON only):
# {{
#   "prompt": "input_prompt",
#   "option": "list contain some options",
#   "answer": "correct answer"
# }}
# """

#     # Call the model to generate response
#     response = client.models.generate_content(
#         model=model,
#         contents=prompt
#     )

#     qa_text = response.text.strip()
#     qa_text = re.sub(r"^```json|```$", "", qa_text).strip()

#     try:
#         qa = json.loads(qa_text)
#     except json.JSONDecodeError:
#         print(f"Could not parse JSON for file {chunk}.")
#         continue

#     qa["file_name"] = chunk.split("/")[-1]
#     results.append(qa)

#     print(qa)
#     time.sleep(5)


In [ ]:
results

In [ ]:
len(results)

In [ ]:
output_path = "/content/diseaseonly25f.json"
with open(output_path, "w", encoding="utf8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

In [ ]:
len(results)

In [ ]:
# Display a few examples of the generated question-answer pairs
for i in range(min(5, len(results))):
    print(f"Question: {results[i]['question']}")
    print(f"Answer: {results[i]['answer']}")
    print("-" * 20)